In [2]:
# -*- coding: utf-8 -*-

#声明Notebook类型，必须在引入pyecharts.charts等模块前声明
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from pyecharts import options as opts
from pyecharts.charts import Page, Radar
from impala.dbapi import connect

#连接Hadoop数据库
v1 = []
v2 = []
v3 = []
conn = connect(host='192.168.1.7', port=10000, database='sales',auth_mechanism='NOSASL',user='root')
cursor = conn.cursor()

#读取Hadoop表数据
sql_num = "SELECT region,ROUND(SUM(sales)/10000,2),ROUND(SUM(profit)/10000,2) FROM orders WHERE dt=2019 GROUP BY region"
cursor.execute(sql_num)
sh = cursor.fetchall()
for s in sh:
    v1.append(s[0])
    v2.append(s[1])
    v3.append(s[2]) 

#画雷达图
def radar_base() -> Radar:
    c = (
        Radar()
        .add_schema(
            schema=[
                opts.RadarIndicatorItem(name="华东", max_=200),
                opts.RadarIndicatorItem(name="华南", max_=200),
                opts.RadarIndicatorItem(name="东北", max_=200),
                opts.RadarIndicatorItem(name="中南", max_=200),
                opts.RadarIndicatorItem(name="西南", max_=200),
                opts.RadarIndicatorItem(name="西北", max_=200),
            ]
        )
        .add("销售额", [v2])
        .add("利润额", [v3])
        .set_global_opts(
            title_opts=opts.TitleOpts(title="区域销售额与利润额的比较分析", subtitle="2019年企业经营状况"),
            legend_opts=opts.LegendOpts(selected_mode="single"),
            toolbox_opts=opts.ToolboxOpts()
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    )
    return c

#第一次渲染时候调用load_javasrcript文件
radar_base().load_javascript()
#展示数据可视化图表
radar_base().render_notebook()